In [3]:
%load_ext tensorboard.notebook

ModuleNotFoundError: No module named 'tensorboard.notebook'

In [1]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


# Data Generator
---

In [6]:
from math import ceil, floor
from random import shuffle
from keras.utils import Sequence

import matplotlib.pyplot as plt
import csv
import numpy as np
from sklearn.model_selection import train_test_split


theta       = 0.35
corrections = {'center': 0, 'left': theta, 'right': -theta}

dx          = 50
shift_val   = {'left': dx, 'right': -dx}


def shift(img, dx):
        
    shifted_img = img.copy()
    if dx > 0:
        # shift right
        shifted_img[:,dx:] = img[:,:-dx]
    else:
        # shift left
        shifted_img[:,:dx] = img[:,-dx:]

    return shifted_img


# ref: https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
class DrivingLogSequence(Sequence):
    
    def __init__ (self, driving_log, batch_size=32):
        
        self.driving_log = driving_log
        self.batch_size = batch_size
        
        return
    
    def __len__(self):
        return floor(len(self.driving_log)/self.batch_size)
    
    def add_data(self, img, steering):
        self.images.append(img)
        self.steerings.append([steering])
        return
    
    def __getitem__(self, index):
        
        batch_driving_log = self.driving_log[index:index+self.batch_size]
        self.images = []
        self.steerings = []
        
        for line in batch_driving_log:
            
            center_steering = float(line['steering'])
            
            for camera in corrections:
                
                filename = line[camera]
                
                img = plt.imread(filename).copy()
                steering = center_steering + corrections[camera]
                
                self.add_data(img,           steering)
                self.add_data(np.fliplr(img), -steering)
                
                if camera != 'center':
                    steering  = steering + corrections[camera]
                    shift_img = shift(img, shift_val[camera])
                    
                    self.add_data(shift_img,            steering)
                    self.add_data(np.fliplr(shift_img), -steering)
            
        X = np.array(self.images)
        y = np.array(self.steerings)
        
        return X, y
    
    def on_epoch_end(self):
        shuffle(self.driving_log)
        return
    

def create_generators(test_size=0.2, shuffle=True):
    
    drv_log_folders = ['data_easy_route','data_hard_route']
    drv_log_filename = 'driving_log.csv'

    drv_log = []
    for drv_log_folder in drv_log_folders:
        with open(drv_log_folder + '/' + drv_log_filename) as driving_log_file:
            driving_log_reader = csv.DictReader(driving_log_file)
            for line in driving_log_reader:
                for camera in corrections:
                    line[camera] = drv_log_folder + "/" + line[camera].strip()
                drv_log.append(line)
                
    training, validation = train_test_split(drv_log, test_size=0.2, shuffle=True)                                       
                       
    return DrivingLogSequence(training), DrivingLogSequence(validation)        
                       
                       
driving_log_seq_training, driving_log_seq_validation = create_generators()
                       

    

In [ ]:
    def OLD__getitem__OLD(self, index):
        
        batch_driving_log = self.driving_log[index:index+self.batch_size]
        images = []
        steerings = []
        
        for line in batch_driving_log:
            
            center_steering = float(line['steering'])
            
            for cam_pos, theta in cameras:
                
                filename = line[cam_pos].strip()
                
                img = plt.imread(filename).copy()
                images.append(img)
                steering = center_steering + theta
                steerings.append([steering])
                
                # augment with flipped image
                flip_img = np.fliplr(img)
                images.append(flip_img)
                steerings.append([-steering])
                
                if cam_pos == 'left':
                    # augment with right shift
                    rightshift_img = shift(img, 50)
                    images.append(rightshift_img)
                    steerings.append([steering + theta])

                    # augment with flipped right shift image
                    flip_img = np.fliplr(rightshift_img)
                    images.append(flip_img)
                    steerings.append([-(steering + theta)])
                    
                elif cam_pos == 'right':
                    # augment with left shift
                    leftshift_img = shift(img, -50)
                    images.append(leftshift_img)
                    steerings.append([steering - theta])

                    # augment with flipped left shift image
                    flip_img = np.fliplr(leftshift_img)
                    images.append(flip_img)
                    steerings.append([-(steering - theta)])
            
        X = np.array(images)
        y = np.array(steerings)
        
        return X, y
    


In [ ]:
    
    
driving_log_filename = 

driving_log = []

with open(driving_log_filename) as driving_log_file:
    driving_log_reader = csv.DictReader(driving_log_file)
    for line in driving_log_reader:
        driving_log.append(line)
        
driving_log_filename = 'data/driving_log.csv'

with open(driving_log_filename) as driving_log_file:
    driving_log_reader = csv.DictReader(driving_log_file)
    for line in driving_log_reader:
        driving_log.append(line)
        
        
driving_log_training, driving_log_validation = train_test_split(driving_log, test_size=0.2, shuffle=True)
driving_log_seq_training   = DrivingLogSequence(driving_log_training)
driving_log_seq_validation = DrivingLogSequence(driving_log_validation)        
    

### Old

In [ ]:
import matplotlib.pyplot as plt

image_name = "sample_data/IMG/left_2016_12_01_13_30_48_287.jpg"
img=plt.imread("sample_data/IMG/left_2016_12_01_13_30_48_287.jpg")
shift_img = shift(img, -50)
plt.imshow(shift_img)

#img_copy = np.copy(img)
#img_copy[:,159:162] = [255, 0, 0]


In [ ]:
plt.imshow(img)

In [ ]:
import numpy as np
def shift(img, dx):

    shifted_img = np.zeros_like(img)
    if dx > 0:
        # shift right
        shifted_img[:,dx:] = img[:,:-dx]
    else:
        # shift left
        shifted_img[:,:dx] = img[:,-dx:]

    return shifted_img

shift_img = shift(img, -75)
plt.imshow(shift_img)

In [ ]:
img.shape

In [ ]:
import csv
import numpy as np

driving_log_filename = 'sample_data/driving_log.csv'

center_images = []
steerings = []
with open(driving_log_filename) as driving_log:
    driving_log_reader = csv.DictReader(driving_log)
    for row in driving_log_reader:
        center_images.append(plt.imread('sample_data/' + row['center']))
        steerings.append([float(row['steering'])])
                            
X_train = np.array(center_images)
y_train = np.array(steerings)
    
print(X_train.shape)
print(y_train.shape)


In [ ]:
# adding left and right cameras

import matplotlib.pyplot as plt
import csv
import numpy as np

driving_log_filename = 'sample_data/driving_log.csv'

images = []
steerings = []
theta = 0.35
with open(driving_log_filename) as driving_log:
    driving_log_reader = csv.DictReader(driving_log)
    for row in driving_log_reader:
        
        steering = float(row['steering'])
        
        images.append(plt.imread('sample_data/' + row['center'].strip()))
        steerings.append([steering])
        
        images.append(plt.imread('sample_data/' + row['left'].strip()))
        steerings.append([steering + theta])
        
        images.append(plt.imread('sample_data/' + row['right'].strip()))
        steerings.append([steering - theta])
        
                            
X_train = np.array(images)
y_train = np.array(steerings)
    
print(X_train.shape)
print(y_train.shape)


In [ ]:
 
'''
def generator(lines, batch_size=32):

    driving_log_filename = 'sample_data/driving_log.csv'

    images = []
    steerings = []
    theta = 0.35
    
    with open(driving_log_filename) as driving_log:
        num_lines = 0
        for line in driving_log:
            num_lines += 1

    with open(driving_log_filename) as driving_log:

        driving_log_reader = csv.DictReader(driving_log)
        
        
        print (len(driving_log_reader))

    for row in driving_log_reader:

        steering = float(row['steering'])

        images.append(plt.imread('sample_data/' + row['center'].strip()))
        steerings.append([steering])

        images.append(plt.imread('sample_data/' + row['left'].strip()))
        steerings.append([steering + theta])

        images.append(plt.imread('sample_data/' + row['right'].strip()))
        steerings.append([steering - theta])
    '''
        

In [ ]:
plt.imshow(X_train[45])

In [ ]:
y_train[46]

for y in y_train:
    if y > 0:
        print(y)
        break

# Network Structure
---

In [3]:
from keras.models import Sequential
from keras.layers import Lambda, BatchNormalization, Flatten, Dense 
from keras.layers import Conv2D, Cropping2D, Dropout, MaxPooling2D
import tensorflow as tf


def normalize(rgb):
    '''
    normalize rgb between [-1, 1]
    '''
    
    return (rgb-128.0) / 128.0


# ---loss: 0.0175
model = Sequential()
model.add(Lambda(normalize, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=[(50, 20), (0, 0)]))
model.add(Conv2D(filters=24, kernel_size=5, strides=2, activation='relu'))
model.add(Dropout(0.1))
model.add(Conv2D(filters=36, kernel_size=5, strides=2, activation='relu'))
model.add(Dropout(0.1))
model.add(Conv2D(filters=48, kernel_size=5, strides=2, activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64, kernel_size=3, strides=1, activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64, kernel_size=3, strides=1, activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

'''
model = Sequential()
model.add(Lambda(normalize, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=[(50, 20), (0, 0)]))
model.add(Conv2D(filters=24, kernel_size=5, strides=2, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv2D(filters=36, kernel_size=5, strides=2, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv2D(filters=48, kernel_size=5, strides=2, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=3, strides=1, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=3, strides=1, activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
'''
          

Using TensorFlow backend.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


"\nmodel = Sequential()\nmodel.add(Lambda(normalize, input_shape=(160,320,3)))\nmodel.add(Cropping2D(cropping=[(50, 20), (0, 0)]))\nmodel.add(Conv2D(filters=24, kernel_size=5, strides=2, activation='relu'))\nmodel.add(Dropout(0.2))\nmodel.add(BatchNormalization())\nmodel.add(Conv2D(filters=36, kernel_size=5, strides=2, activation='relu'))\nmodel.add(Dropout(0.2))\nmodel.add(BatchNormalization())\nmodel.add(Conv2D(filters=48, kernel_size=5, strides=2, activation='relu'))\nmodel.add(BatchNormalization())\nmodel.add(Conv2D(filters=64, kernel_size=3, strides=1, activation='relu'))\nmodel.add(BatchNormalization())\nmodel.add(Conv2D(filters=64, kernel_size=3, strides=1, activation='relu'))\nmodel.add(BatchNormalization())\nmodel.add(Flatten())\nmodel.add(Dense(100, activation='relu'))\nmodel.add(Dropout(0.5))\nmodel.add(BatchNormalization())\nmodel.add(Dense(50, activation='relu'))\nmodel.add(Dropout(0.5))\nmodel.add(BatchNormalization())\nmodel.add(Dense(10, activation='relu'))\nmodel.add(D

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_10 (Batc (None, 160, 320, 3)       12        
_________________________________________________________________
cropping2d_2 (Cropping2D)    (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 43, 158, 24)       1824      
_________________________________________________________________
batch_normalization_11 (Batc (None, 43, 158, 24)       96        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 20, 77, 36)        21636     
_________________________________________________________________
batch_normalization_12 (Batc (None, 20, 77, 36)        144       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 37, 48)         43248     
__________

In [4]:
model.compile(loss='MSE', optimizer='Adam')

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

# https://keras.io/api/callbacks/early_stopping/
early_stopper = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit_generator(driving_log_seq_training, 
                    validation_data=driving_log_seq_validation, 
                    epochs=30,
                    callbacks=[early_stopper])

model.save('model.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/30



387/387 [==============================] - 142s 367ms/step - loss: 0.0621 - val_loss: 0.1012
Epoch 2/30
387/387 [==============================] - 130s 336ms/step - loss: 0.0185 - val_loss: 0.0667
Epoch 3/30
387/387 [==============================] - 131s 339ms/step - loss: 0.0104 - val_loss: 0.0569
Epoch 4/30
387/387 [==============================] - 131s 339ms/step - loss: 0.0088 - val_loss: 0.0623
Epoch 5/30
387/387 [==============================] - 130s 336ms/step - loss: 0.0080 - val_loss: 0.0479
Epoch 6/30
387/387 [==============================] - 130s 337ms/step - loss: 0.0065 - val_loss: 0.0447
Epoch 7/30
387/387 [==============================] - 133s 344ms/step - loss: 0.0079 - val_loss: 0.0650
Epoch 8/30
387/387 [==============================] - 132s 341ms/step - loss: 0.0060 - val_loss: 0.0350
Epoch 9/30
387/387 [==============================] - 128s 332ms/step - loss: 0.0066 - val_loss: 0.0370
Epoch 10/30
387/387 [==============================] - 132s 340ms/step - lo

In [14]:
model.save('model.h5')

### Old

In [ ]:
model.fit(X_train, y_train, batch_size=512, initial_epoch=0, epochs=4, shuffle=True, validation_split=0.2)

In [ ]:
model.save('model.h5')

# drive.py

In [ ]:
import argparse
import base64
from datetime import datetime
import os
import shutil

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
from PIL import Image
from flask import Flask
from io import BytesIO

from keras.models import load_model
import h5py
from keras import __version__ as keras_version

sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None


class SimplePIController:
    def __init__(self, Kp, Ki):
        self.Kp = Kp
        self.Ki = Ki
        self.set_point = 0.
        self.error = 0.
        self.integral = 0.

    def set_desired(self, desired):
        self.set_point = desired

    def update(self, measurement):
        # proportional error
        self.error = self.set_point - measurement

        # integral error
        self.integral += self.error

        return self.Kp * self.error + self.Ki * self.integral


controller = SimplePIController(0.1, 0.002)
set_speed = 9
controller.set_desired(set_speed)


@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        # The current steering angle of the car
        steering_angle = data["steering_angle"]
        # The current throttle of the car
        throttle = data["throttle"]
        # The current speed of the car
        speed = data["speed"]
        # The current image from the center camera of the car
        imgString = data["image"]
        image = Image.open(BytesIO(base64.b64decode(imgString)))
        image_array = np.asarray(image).copy()
        image_array[:,159:162] = [255, 0, 0]                
        steering_angle = float(model.predict(image_array[None, :, :, :], batch_size=1))

        throttle = controller.update(float(speed))

        print(steering_angle, throttle)
        send_control(steering_angle, throttle)

        # save frame
        if args.image_folder != '':
            timestamp = datetime.utcnow().strftime('%Y_%m_%d_%H_%M_%S_%f')[:-3]
            image_filename = os.path.join(args.image_folder, timestamp)
            image.save('{}.jpg'.format(image_filename))
    else:
        # NOTE: DON'T EDIT THIS.
        sio.emit('manual', data={}, skip_sid=True)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit(
        "steer",
        data={
            'steering_angle': steering_angle.__str__(),
            'throttle': throttle.__str__()
        },
        skip_sid=True)


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Remote Driving')
    parser.add_argument(
        'model',
        type=str,
        help='Path to model h5 file. Model should be on the same path.'
    )
    parser.add_argument(
        'image_folder',
        type=str,
        nargs='?',
        default='',
        help='Path to image folder. This is where the images from the run will be saved.'
    )
    #parser.parse_args()    
    #parser.parse_args(['--sum', '7', '-1', '42'])    
    args = parser.parse_args(['model.h5'])

    # check that model Keras version is same as local Keras version
    f = h5py.File(args.model, mode='r')
    model_version = f.attrs.get('keras_version')
    keras_version = str(keras_version).encode('utf8')

    if model_version != keras_version:
        print('You are using Keras version ', keras_version,
              ', but the model was built using ', model_version)

    #mine
    print(args.model)
    model = load_model(args.model)

    if args.image_folder != '':
        print("Creating image folder at {}".format(args.image_folder))
        if not os.path.exists(args.image_folder):
            os.makedirs(args.image_folder)
        else:
            shutil.rmtree(args.image_folder)
            os.makedirs(args.image_folder)
        print("RECORDING THIS RUN ...")
    else:
        print("NOT RECORDING THIS RUN ...")

    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)


In [ ]:
import sys
print(sys.version)


In [ ]:
import socketio